In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import requests
import time
import json
!pip install tweepy
import tweepy
!pip install wptools
import wptools
from PIL import Image
from io import BytesIO

1. Directly download weratedogs twitter archive .csv file

In [2]:
tweets = pd.read_csv("../input/weratedogs-twitter-archive-enhanced/twitter-archive-enhanced.csv")
tweets.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


2. Download the tweet image prediction .tsv file using url

In [3]:
folder = 'tweet_tsv'
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

if not os.path.exists(folder):
    os.makedirs(folder)
    
r = requests.get(url)
with open(os.path.join(folder, url.split("/")[-1]), mode='wb') as file:
    response = file.write(r.content)

In [4]:
image = pd.read_csv('./tweet_tsv/image-predictions.tsv', sep='\t')
image.head(1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


3. Use tweepy library to query additional data via the Twitter API
(tweet_json.txt):
    * Tweepy (twitter api) allows us to mine the data of any twitter user. Using .get_status of the API class in tweepy module fetch a status(tweet)

In [ ]:
consumer_key = 'mP9YUkyksJ3gGBcmLWWNjG1HK'
consumer_secret = 'O9joKKsymUtGXMg0rsUsr4uKm3J44y5nfadefJHm3sDWBget0b'
access_token = '1577688773621694465-rWwJYImL2LRjB37xJAHr51cxSECQ9G'
access_token_secret = '9Zvzb6dNMmAe8wTI9Ppzcsv7z63aRX5MgspDP9hHJOS0M' 

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
# add `wait_on_rate_limit` parameter to automatically wait for rate limits to replenish
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True)

In [ ]:
start_time = time.time()

# Save all JSON data of each tweet and save the queries into a 'tweet_json.txt' file 
# so that you only need to run the following long queries once.
with open ('tweet_json.txt', 'w') as file:
    for tweet_id in tweets['tweet_id']:
        try:
            # Get status/tweet for each tweet_id
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            # Convert into .json string using .dumps() function 
            # Also, add '\n' as each tweet's JSON data to be written to its own line
            file.write(json.dumps(tweet) + "\n") 
        except Exception as e:
            # if error occurs, print tweet_id and error message
            print("No tweet found for {} with error message: {}".format(str(tweet_id), str(e)))
            
end_time = time.time()
print("The process finished in {} seconds".format(end_time - start_time))

In [ ]:
# Read JSON line by line and create a python dictionary
info = []

with open('tweet_json.txt', 'r') as json_file:
    for line in json_file:
        # Parse JSON encoded/formatted string line by line
        json_data = json.loads(line)
        # Create a dictionary
        info.append({'tweet_id': json_data['id'], #call the value by stating 'key'
                    'favorites': json_data['favorite_count'],
                     'retweets': json_data['retweet_count'],
                     'timestamp': json_data['created_at']
                    })

# Create a dataframe
additional = pd.DataFrame(info, columns=['tweet_id', 'favorites', 'retweets', 'timestamp'])

# Optional: Save the dataframe to .csv and add the data into this notebook.
# Doing so, you don't have to use twitter api everytime you reboot this notebook, which takes a significant amount of time. .
# additional.to_csv('additional.csv', index=False)

In [5]:
additional = pd.read_csv("./additional.csv")
additional.head(1)

,tweet_id,favorites,retweets,timestamp
0,892420643555336193,33594,6950,Tue Aug 01 16:23:56 +0000 2017


### Quality issues
##### tweets
1. Drop retweets rows (for all tables)
2. More rows than image (indicating there are tweets without pictures)
3. Other forms of Nan values(e.g. "None") 미

##### image_predict
1. Inconsistencies in upper/lowercase in `p1`, `p2`, `p3` 
2. Inconsistencies in upper/lowercase in dog breed predictions
3. Inconsistencies in the length of decimal numbers: `p1_conf`, `p2_conf`, `p3_conf`

##### additional
1. drop retweet rows
2. `rating_denominator` should be 10

### Tidiness issues 
##### tweets table
1. Unnecessary columns that could be joined into one `stage` column with possible values of doggo, floofer, pupper, or puppo
2. Unnecessary columns with too many null values: `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`,`retweeted_status_user_id`,`retweeted_status_timestamp`.
3. Wrong datatypes `tweet_id`, `timestamp`  

##### image table
1. A column needed for actual dog breed, not falsely predicted object 
2. Wrong datatype `tweet_id`

##### additional table
1. Wrong data types of `tweet_id`, `timestamp` columns. Ignore `timestamp` for now as we will drop the column later.
2. Duplicate `timestamp` information in both df1 and df2 datasets

In [7]:
# Before any cleaning, make a copy of original datasets.
tweets_clean = tweets.copy()
image_clean = image.copy()
add_clean = additional.copy()

##### Clean 1 (all tables)
Define: 
- Drop retweet cases from all tables as this project will only analyse original ratings that have image (no retweets)

Code 1: merge tables

In [ ]:
# Merge `text` column to image and additional table on='tweet_id'.
# To do so, first, create a separate table with identifier column `tweet_id` and `text`.
texts = tweets_clean[['tweet_id', 'text']]

# Merge
image_clean = pd.merge(image_clean, texts, on='tweet_id')
add_clean = pd.merge(add_clean, texts, on='tweet_id')

# Confirm
image_clean.columns, add_clean.columns

Code 2: remove retweets

In [ ]:
# First, define a function that returns a mask of retweet entries.
def rt_mask(table):
    mask = table.text.str.contains("RT")
    return mask

# Drop retweet rows using the function.
tweets_clean = tweets_clean[~rt_mask(tweets_clean)]
image_clean = image_clean[~rt_mask(image_clean)]
add_clean = add_clean[~rt_mask(add_clean)]

In [ ]:
# Confirm if there is any RT left in tables. You should get False's.
print(tweets_clean[tweets_clean['text'].str.contains('RT')].sum().any()),
print(image_clean[image_clean['text'].str.contains('RT')].sum().any()),
print(add_clean[add_clean['text'].str.contains('RT')].sum().any())

In [ ]:
# Once it is confirmed, drop the redudant `text` columns from previously merged tables.
image_clean.drop('text', axis=1, inplace=True)
add_clean.drop('text', axis=1, inplace=True)

# Confirm. Column `text` should not be included in a duplicated series.
all_col = pd.Series(list(tweets_clean) + list(image_clean) + list(add_clean))
all_col[all_col.duplicated()]

##### Clean 2 (all tables) 
Define: 
- More rows than image (indicating there are tweets without pictures) 

Code 1:

In [ ]:
# Extract `tweet_id` from image_clean and make it into a list
id_list = list(image_clean.tweet_id)

# Then, query only the rows with `tweet_id` in image_clean table.
tweets_clean = tweets_clean[tweets_clean['tweet_id'].isin(id_list)]
add_clean = add_clean[add_clean['tweet_id'].isin(id_list)]

# If you want to replace the two above lines with a defined function, use below code.
# def id_select(table, varName='tweet_id'): # arguments with '=' are set as a default
    # table = table[table[varName]].isin(id_list)
    # return table

In [ ]:
# Confirm with boolean statement. You should get two True's
print(tweets_clean.tweet_id.isin(image_clean.tweet_id).sum().any(),
add_clean.tweet_id.isin(image_clean.tweet_id).sum().any())

> self-study note: <br>

when creating a function,
1. Avoid using actual dataframe names in your function - the placeholder can be replaced with an actual dataframe name later (and python can actually use global variables inside functions, so it is error prone)
2. Including optional arguments, like varName, allows you to use the function for different columns while, at the same time, using the default when you call the function.
3. Using a return statement allows you to use a different dataframe name for the result. This avoids over-writing already existing dataframes.

##### Clean 2 (tweets_clean)
Define:
Drop unnecessary columns with too many null values

Code:

In [ ]:
# Make a list of columns to drop
drop_cols = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp']

# Drop
tweets_clean.drop(columns = drop_cols, inplace=True)

# Confirm
tweets_clean.info()

##### Clean 3 (tweets_clean)
Define: 
Join redundant columns into `stage` column with possible values of doggo, floofer, pupper, or puppo

Code:

In [ ]:
# By running the code below, you can see that other than empty space or actual stage, "None" string value is included in the column value.
print("doggo: {}, floofer: {}, puppo: {}, pupper:{}"
     .format(tweets_clean.doggo.unique(), 
            tweets_clean.floofer.unique(),
            tweets_clean.puppo.unique(),
            tweets_clean.pupper.unique()))

In [ ]:
# So before joining the four columns, first replace "None" to empty-space "".
# Then join the columns.
# Some columns for example will be "   floofer ", reflecting empty spaces created by joining the columns. So strip those empty spaces.
tweets_clean['stage'] = (tweets_clean[['doggo', 'floofer', 'puppo', 'pupper']]
                         .replace("None","").apply(lambda x: " ".join(x), axis=1).str.strip())

# Confirm if merge successful.
tweets_clean.query("stage != ''").head(1)

In [ ]:
# Drop the redundant columns
tweets_clean.drop(columns = ['doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

# Confirm
tweets_clean.dtypes

##### Clean 3 (all tables)
Define: 
- Convert wrong datatypes

Code:

In [ ]:
# Create a function that converts wrong datatypes with input arguments (table)
def change_dtypes(table):
    for col in table.columns:
        if col[-3:] == "_id":
            table[col] = table[col].astype(str)
        elif col[-5:] == "stamp":
            table[col] = pd.to_datetime(table[col])
        else:
            table[col] = table[col]
            
# Call the function
change_dtypes(tweets_clean)
change_dtypes(image_clean)
change_dtypes(add_clean)

In [ ]:
# Confirm
tweets_clean.dtypes, image_clean.dtypes, add_clean.dtypes

##### Clean 4 (image_clean)
Define: 
- Create an new column for actual breed of a dog, not falsely predicted breed.

Code:
- np.where(condition, value if condition True, value if condition False)
- Since the prediction p1, p2, p3 is hierarchical, if the first prediction is True (i.e. is a dog), that is the prediction you want. If False (i.e. not a dog), move onto the second prediction
- `px_dog` is vectorised, so you can just use the column itself as a condition in np.where() 

In [ ]:
image_clean['dog_breed'] = np.where(image_clean['p1_dog'], image_clean['p1'],
                                   np.where(image_clean['p2_dog'], image_clean['p2'],
                                           np.where(image_clean['p3_dog'], image_clean['p3'],
                                           np.nan)))

In [ ]:
# Confirm
image_clean.sample(3)

##### Clean 5 (all tables)
Define: 
- Drop any duplicate columns amongst tables

Code:

In [ ]:
# Look for duplicated columns
all_cols = pd.Series(list(tweets_clean) + list(image_clean) + list(add_clean))
all_cols[all_cols.duplicated()]

In [ ]:
# Drop `timestamp` column from `add_clean` table as it is irrelevant to table's observational unit
add_clean.drop('timestamp', axis=1, inplace=True)

# Confirm
add_clean.columns

##### Clean 6 (image_clean)
Define: 
- Lowercase all breed values in `p1`, `p2`, `p3`, `dog_breed` columns
- Lowercase `dog_breed` column 
- Standardise the length of decimal numbers in `p1_conf`, `p2_conf`, `p3_conf`: .round()

Code 1:

In [ ]:
# Lowercase
def lowercase(table):
    breed_cols = ['p1', 'p2', 'p3', 'dog_breed']
    for c in table.columns:
        if c in breed_cols:
            table[c] = table[c].str.lower()

# Call the function
lowercase(image_clean)

# Confirm
image_clean.sample(5)

Code 2:

In [ ]:
# Round decimal numbers
for c in image_clean.columns:
    if c[-4:] == "conf":
        image_clean[c] = image_clean[c].round(3)

# Confirm
image_clean.head(2)

##### Clean 7(add_clean)
Define: 
- Replace invalid rating_denominator with 10

Code:

In [ ]:
# Create a mask for`rating_denominator` value other than 10.
mask = tweets_clean['rating_denominator'] != 10

# Then assign the value of 10 to those wrong entries.
tweets_clean.loc[mask, 'rating_denominator'] = 10

# Confirm
tweets_clean.query('rating_denominator != 10')

##### Clean 8 (tweets_clean) 
Define: 
- Replace non-name values (e.g. None, very) to Nan 

Code:

In [32]:
# Convert invalid names to NaN: np.nan()
non_name = ['None', 'very', 'incredibly', 'his', 'just', 'getting', 'mad', 'this', 'unacceptable', 'all', 
            'old', 'by', 'life', 'light', 'space', 'a']
        

for n in tweets_clean['name']:
    if n in non_name:
        n = None

tweets_clean[tweets_clean['name']=='None']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
